### Query ESO archive and calculate total time for each PID
#### in any given time frame

In [3]:
import astroquery
from astroquery.eso import Eso
import pandas as pd
import numpy as np
import sys
import math
from numpy import *
from astropy.table import Table
from astropy.table import *
from astropy.io import ascii
from datetime import datetime, date, timedelta
from termcolor import colored

eso=Eso()
eso.ROW_LIMIT = -1 


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)

### Defines a general search for available data in the ESO archive
#### - using  a date range, PIDs and instrument as input

In [4]:
def eso_query(instr,start,end,pid):
    num_obs= 0
    tot_exptime = 0.0
    table = eso.query_main(column_filters={'instrument':instr,'dp_cat': 'SCIENCE','stime':start,'etime':end,'prog_id':pid})
    if not table:
        num_Obs      = num_obs + 0
        tot_exptime  = 0.0
    else:
        num_Obs = num_obs + len(table)
        pd_table    = table.to_pandas()
        tot_exptime = pd_table["Exptime"].sum()
    
    return num_Obs,tot_exptime


def eso_query_target(instr,target,start,end,pid):
    num_obs= 0
    tot_exptime_ir = 0.0
    tot_exptime_ir = 0.0
    table = eso.query_main(column_filters={'instrument':instr,'target':target_,'dp_cat': 'SCIENCE','stime':start,'etime':end,'prog_id':pid})
    if not table:
        num_Obs      = num_obs + 0
        tot_exptime  = 0.0
    else:
        num_Obs = num_obs + len(table)
        pd_table    = table.to_pandas()
        tot_exptime = pd_table["Exptime"].sum()
    
    return num_Obs,tot_exptime


def eso_query_grond(instr,start,end,pid):
    num_ir = 0
    num_opt=0
    num_obs = 0
    table = eso.query_main(column_filters={'instrument': instr, 'prog_id':pid,'dp_cat': 'SCIENCE','stime':start,'etime':end} )
        
    if not table: 
        num_opt = 0
        num_nir  = 0
        tot_exptime_ir = 0.0
        tot_exptime_opt = 0.0
        tot_time = 0.0
        
    else:
# Check for optical images        
        lambda_lo = 1100
        table_opt = table[table['filter_lambda_min'] < lambda_lo] # limit list to only optical files
        pd_table  = table_opt.to_pandas()
        num_opt   = len(table_opt)                                     # number of optical images
        tot_exptime_opt = (pd_table["Exptime"].sum())
        table_nir      = table[table['filter_lambda_min'] > lambda_lo]      # number of near-infrared images
        pd_table       = table_nir.to_pandas()
        tot_exptime_ir = (pd_table["Exptime"].sum())
        num_nir = len(table_nir)
        num_obs = num_opt + num_nir
#        print(pid_1, "   ",num_nir, "    ",tot_exptime_ir)
    return num_opt,num_nir,tot_exptime_ir, tot_exptime_opt


In [18]:

info = "/home/angela/LaSilla/P115/P115_pi.list"

data = open(info,'r')

#start   = input('Start date of query (yyyy-mm-dd):  ')
#end     = input('End date of query (yyyy-mm-dd):    ')
start = "2025-04-01"

end = (datetime.now()+timedelta(days=2)).strftime('%Y-%m-%d')             # date for tomorrow, to make sure that all nights are  used
#end = "2025-05-28"


text = colored(end, "red")

print("Statistics until: ",text, "to make sure all data are found")   

end_f   = (datetime.now() + timedelta(days=-1) ).strftime('%Y-%m-%d')     # date for yesterday, the day the last observing night started
#end_f   = end.strftime('%Y-%m-%d')

print("     ")
print("     ")
print("     ")

orig_stdout = sys.stdout

path = "/home/angela/LaSilla/P115"

file_out   = path + "/Statistic/P115_stats.cvs"
file_out_2 = path + "/Statistic/P115_stats_a.cvs"

data_2 = []
col1=[]    # full name
col2=[]    # list of PIDs
col3=[]    # instrument
col4=[]    # allocated time in hours Category A
col5=[]    #  requested time in proposal
col6=[]    # number of files
col7=[]    # exposure times
col8=[]    # execution time in hr
col9=[]    # fraction in %
col10 =[]  # fraction completed in of total %


data_3 = []
cola=[]    # full name
colb=[]    # 1st PID
colc=[]    # 2nd PID
cold=[]    # instrument
cole=[]    # allocated time A in hr
colf=[]    # tie asked for in proposal
colg=[]    # number of files
colh=[]    # exposure time in hr
coli=[]    # observed time in hr
colj=[]    # fraction completed in %
colk=[]    # fraction of total time asked for in proposal

num_DDT     = 0
num         = 0
tot_exptime = 0.0
header1 = data.readline()
header2 = data.readline()

for line in data:    
    num = 0
    tot_exptime = 0.0
    tot_time    = 0.0
    
    line     = line.strip()
    columns  = line.split()
    name     = columns[0]
    first    = columns[1]
    email    = columns[2]
    pid_1    = columns[3]
    pid_2    = columns[4]
    instr    = columns[5]
    t_tot_A  = float(columns[6])    
    t_tot_B  = float(columns[7])
    tot_alloc_time = t_tot_A  + t_tot_B 
#    print(name,tot_alloc_time)
    f_name = name+','+ first
    full_name = ("{:24}".format(f_name))

    idlist = pid_1
    if pid_2 != str("000.0000.000"):
        idlist = idlist+", "+pid_2
        


# define overheads per image    
    if instr  == "FEROS":
        
        result_1 = eso_query(instr,start,end,pid_1)
        num = num+result_1[0]
        tot_exptime= tot_exptime+result_1[1]

        result_2 = eso_query(instr,start,end,pid_2)
        num= num + result_2[0]
#        print(num)
        tot_exptime = tot_exptime+result_2[1]
        oh_time = 282.0
        tot_exptime = tot_exptime/3600.0
        tot_time    = ((num*oh_time)/3600.0)  + tot_exptime

        if name== "COSTA":                               # usual does 2 exposures in 1 OB, overheads are 05:24 min=324 sec
            num=0
            tot_exptime= 0.0
            result_1 = eso_query(instr,start,end,pid_1)
#            print(result_1[0])
            num = num+result_1[0]
            tot_exptime= (tot_exptime+result_1[1])
            result_2 = eso_query(instr,start,end,pid_2)
            num = num+result_2[0]
            tot_exptime= (tot_exptime+result_2[1])/3600.0

            tot_time = ((num/2)*(324.0)/3600.0)+(tot_exptime)

        if name== "KORHONEN":                            # usual does 3 exposures in 1 OB, overheads are 06:06 min=366 sec
            num=0
            tot_exptime= 0.0
            overhead = 366.0                             # for completed OBs, each OB has 3*100 sec exposures
            result_1 = eso_query(instr,start,end,pid_1)
#            print(result_1[0])
            num = num+result_1[0]                       # only counts complete OBs, each with 3*100 sec exposures
#            print("Number ",num)
            tot_exptime= ((tot_exptime+result_1[1]))/3600
            tot_time = ((overhead*math.trunc((num/3)))/3600.0)+tot_exptime

            
#        print("Finished  ",name)
    if instr == "WFI" :
        tot_time= 0.0
        if name =="BANADOS":
            
            result_1      = eso_query(instr,start,end,pid_1)
            num           = num + result_1[0]
            tot_exptime   = (tot_exptime +result_1[1]) /3600.00
            num_tot       = math.floor(num/5)
            over_heads    = (num_tot*1130)                                  #  1130 sec overheads for each 5 exposures!!!
            tot_time      = tot_exptime+(over_heads/3600.0)
            
        if name =="NEUMANN":
            num=0
            tot_exptime = 0.00
            
            result_1    = eso_query(instr,start,end,pid_1)
            num         = num+result_1[0]
            tot_exptime = ((tot_exptime + result_1[1]))/3600.0
            tot_time    = tot_exptime + ((num*90.0)/3600.0)
         
        
        if name=="RAU":
            num=0
            tot_exptime = 0
            
            result_1    = eso_query(instr,start,end,pid_1)
            num         = num+result_1[0]
            overheads   =   (num*150.0)       # 19 min overheads per OB (1 f
            tot_exptime = (tot_exptime+result_1[1])/3600.0
            tot_time    = tot_exptime+(overheads)/3600.0

        if name == "SUYU":
            overhead    =  980.0             # new overhead, 3.5 min for fetch... + setup + readout
            num         = 0
            tot_exptime = 0.0
            result_1    = eso_query(instr,start,end,pid_1)
            num         = num+result_1[0]
            tot_exptime = (tot_exptime+result_1[1])/3600.0
            result_2    = eso_query(instr,start,end,pid_2)
            num         = num+result_2[0]
            tot_exptime = (tot_exptime+result_2[1])/3600.0
           
            tot_time    = tot_exptime+((num/8)*overhead)/3600.0

           

########GROND
# output of eso_query_grond: num_opt,num_nir,tot_exptime_ir, tot_exptime_opt
    
    if instr == "GROND":
        num_op        = 0
        num_ir        = 0
        overhead      = 240.0                                              # overheads per OB are now 3.5 min, 210 sec
        tot_exptime   = 0.0                                                # exposure time in optical
        tot_time      = 0.0                                                # execution time#

        result_1      = eso_query_grond(instr,start,end,pid_1)
        num           = num + result_1[0]+result_1[1] 
        tot_exptime   = tot_exptime + result_1[2]
        
    

#        tot_time = (tot_exptime+ (num *overhead)                             # for GROND overheads are basically = 0

                    
        if name=="DEMIANENKO":
            start_dem= "2025-05-01"
            num          = 0
            tot_exptime  = 0
            result_1      =   eso_query_grond(instr,start_dem,end,pid_1)
            num           =   num + result_1[0]+result_1[1]               # counts optical and near-infrared images separatly
            tot_exptime   = (tot_exptime + result_1[3]) / 3600.0          # uses optical exposure time to calculate total execution time
            tot_time      = tot_exptime + ((num*240.0)/3600.0)

        if name=="BRANDNER":                                            # uses only NIR images, optical images with 1 sec are not counted
            num= 0
            num_tot = 0
            tot_exptime = 0.0
            result_1    = eso_query_grond(instr,start,end,pid_1)
            num_ir      = num + result_1[1] # number of NIR images, optical is not used
#            print("Number of optical exposures  (1 sec): ",result_1[0])
#            print("Number of NIR exposures",result_1[1])
            tot_exptime = (result_1[1]*41.0) / 3600.0                                   # 41 sec per IR exposure
#            print(tot_exptime)
            tot_time    = ((((num_ir/24)*overhead))/3600.0)+tot_exptime
#            print("Total execution time [h] =", tot_time)           
            num  = num_ir
            
        if name=="RAU":
            overhead    = 260.0             # new overhead, 3.5 min for fetch... + setup + readout
            num         = 0
            num_ir      = 0
            num_opt     = 0
            tot_exptime = 0.0
            result_1    = eso_query_grond(instr,start,end,pid_1)
            num_opt     = num_opt + result_1[0] # number of opt images, optical is not used
            num_ir      = num_ir  + result_1[1]
#            print("Number of optical exposures ",result_1[0])
#            print("Number of NIR exposures",result_1[1])
            tot_exptime = (tot_exptime + result_1[3])/3600.0                                  # 41 sec per IR exposure
#            print(tot_exptime)
            tot_time    = ((((num_opt/6)*overhead)+ (num_opt*46))/3600.0) + tot_exptime
#            print("Total execution time [h] =", tot_time)
            num         = num_opt+num_ir      
       
         
            
#### Calculate percentages of allocated times

    time_perc_A = 0.0                                                       # percentage of Cat A. time observed
    time_perc_B = 0.0                                                       # percentage of Cat B. time observed
    
# Case 1: only category A is assigned, and total execution time is greater than 0.0:
    
    if tot_time > 0.0 and t_tot_A > 0.0 and t_tot_B == 0.0:
        time_perc_A = (100.0* (tot_time /t_tot_A))
        time_perc_B = 0.0
    elif tot_time > 0.0 and t_tot_A > 0.0 and t_tot_B > 0.0 and tot_time < t_tot_A:   # time observed is less than Cat. A
        time_perc_A = (100.0* (tot_time /t_tot_A))
        

# Case 2: only category B is assigned, and total execution time is greater than 0.0:        
    elif  tot_time > 0.0 and t_tot_B > 0.0 and t_tot_A == 0.0:  
        time_perc_B = 100.0* (tot_time /t_tot_B)
        time_perc_A = 0.0

# Case 3: both category A & B are assigned, and the total execution time is greater than the time assigned to Category A:    
    elif tot_time > t_tot_A and t_tot_B > 0.0  and t_tot_A !=0: 
        time_perc_A = 100.0
        time_perc_B = (100.0* (tot_time /t_tot_A))-100.0

# Case 4: no time has been assigned to either category (not useful):    
    elif t_tot_A == 0 and t_tot_B == 0:
        time_perc_A = 0.0
        time_perc_B = 0.0

# Case 5: no observation has been done:
    elif tot_exptime  == 0.0:
        time_perc_A   = 0.0
        time_perc_B   = 0.0




### Format the values
    
    t_tot_A_f      = ("{:>6.0f}".format(t_tot_A))                            # formatted allocated time cat- A
    t_tot_B_f      = ("{:>6.0f}".format(t_tot_B))                            # formatted allocated time cat- B
    tot_time_f     = ("{:>6.2f}".format(tot_time))                           # formatted observed time
    time_perc_A_f  = ("{:>6.2f}".format(time_perc_A))
    time_perc_B_f  = ("{:>6.2f}".format(time_perc_B))
    tot_exptime_f  = ("{:>6.2f}".format(tot_exptime))
#    print(name,num, tot_time_f,tot_exptime_f)

    data_2.append({'#PI_name                ': full_name.ljust(20," "),
                   'PIDs              ': idlist.ljust(30," "),
                   'Inst.': str(instr).center(5," "),
                   'Cat-A [h]': str(t_tot_A_f).rjust(10," "),
                   'Cat-B [h]': str(t_tot_B_f).rjust(10," "),
                   '# files': str(num).rjust(10," "),
                   'exp. [h]': tot_exptime_f,
                   'exec. [h] ': str(tot_time_f).center(10," "),
                   '  A [%]': time_perc_A_f,
                   '  B [%]': time_perc_B_f
        }              
    )
    col1.append(full_name)
    col2.append(idlist.ljust(30," "))
    col3.append(instr.center(9," "))
    col4.append(t_tot_A_f)
    col5.append(t_tot_B_f)
    col6.append(num)
    col7.append(tot_exptime_f)
    col8.append(tot_time_f)
    col9.append(time_perc_A_f)
    col10.append(time_perc_B_f)
              
    data_3.append({'#PI_name': full_name.ljust(20," "),
                   '       PID_1       ': pid_1.center(15," "),
                   '       PID_2       ': pid_2.center(15," "),
                   'Instrument  ': str(instr).center(5," "),
                   'Cat-A [h]': str(t_tot_A_f).rjust(10," "),
                   'Cat-B [h]': str(t_tot_B_f).rjust(10," "),
                   '# of files': str(num).rjust(10," "),
                   'exp. [h]': tot_exptime_f,
                   'exec.[h] ': str(tot_time_f).center(10," "),
                   '  A [%]': time_perc_A_f,
                   '  B.[%]': time_perc_B_f
        }              
    )
    cola.append(full_name)
    colb.append(pid_1.center(15," "))
    colc.append(pid_2.center(15," "))
    cold.append(instr.center(15," "))
    cole.append(t_tot_A_f)
    colf.append(t_tot_B_f)
    colg.append(num)
    colh.append(tot_exptime_f)
    coli.append(tot_time_f)
    colj.append(time_perc_A_f)
    colk.append(time_perc_B_f)


pd.set_option('display.precision', 1)    
df = pd.DataFrame(data_2)
df_1 = pd.DataFrame(data_3)
number = len(df.index)
#print(number)
print(df.to_string(index=False))
#print(new)
li = [df.columns.values.tolist()] + df.values.tolist()
df.to_csv(file_out, quoting=None,index=False) 
df_1.to_csv(file_out_2, index=False) 
print("    ")
print("    ")

print("Finished")
sys.stdout=orig_stdout


Statistics until:  2025-06-29 to make sure all data are found
     
     
     


#PI_name                             PIDs               Inst.  Cat-A [h]  Cat-B [h]    # files exp. [h] exec. [h]    A [%]   B [%]
RAU,Arne                 114.27S7.001                   GROND        142          0        617     6.79     8.88      6.25    0.00
DEMIANENKO,Mariia        115.28KF.001                   GROND         72          0        479     8.62    40.55     56.32    0.00
SUYU,Sherry              115.28JZ.003                   GROND        118          0          0     0.00     0.00      0.00    0.00
BRANDNER,Wolfgang        115.28KJ.001                   GROND         27          0        432     4.92     6.12     22.67    0.00
RAU,Arne                 114.27SB.001                   FEROS         20          0          3     2.33     2.57     12.84    0.00
APPELÁNIZ,Jesús          115.28JD.001                   FEROS        135          0        399    88.82   120.07     88.94    0.00
SEEBURGER,Rhys           115.28JC.001                   FEROS         15          0

### Detailed search, also including corrections for:

#### - adjusted for GROND (W.Brandner only uses IRACE)
#### - specific overheads for WFI programs with time-consuming filter changes, e.g. Bañados
#### - M. Demianenko not included yet!!!

### Write a pdf file with results

In [8]:
import reportlab
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter,A4
from reportlab.lib.units import inch
from reportlab.lib import colors
import os
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle,Paragraph,Spacer
from reportlab.lib.styles import ParagraphStyle,getSampleStyleSheet
from reportlab.lib.units import cm

styles = getSampleStyleSheet()

path = "/home/angela/LaSilla/P115/Statistic/"
date_now = (datetime.now()).strftime('%Y-%m-%d')
pdf_out = path + end_f+".pdf"
print(pdf_out)
doc = SimpleDocTemplate(pdf_out, pagesize=A4,fontsize=8)
# container for the 'Flowable' objects
elements = []



colw = (1*inch,1.2*inch,0.55*inch,0.6*inch, 0.6*inch, 0.6*inch, 0.6*inch,0.6*inch, 0.6*inch,0.6*inch)
t=Table(li,colw,(number+1)*[0.3*inch])

t.setStyle(TableStyle([('ALIGN',(1,1),(8,number),'CENTER'),
                       ('ALIGN',(1,1),(1,number),'LEFT'),
                       ('FONTSIZE',(0,0),(10,number),5),
                       ('TEXTCOLOR',(1,1),(10,number),colors.black),
                       ('TEXTCOLOR',(0,0),(0,-1),colors.blue),
                       ('BACKGROUND', (0, 0), (0, number), colors.lightblue),
                       ('BACKGROUND', (1, 0), (10, 0), colors.lightblue),
                       ('BACKGROUND',(0,number-2),(0,number),colors.orange),
                       ('VALIGN',(0,-1),(-1,-1),'MIDDLE'),
                       ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                       ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                       ]))
elements.append(t)


title = "P115 from " + start + " until "+end_f

flowables = [
    Paragraph(title, styles['Title']),
    t,
    Spacer(1 * cm, 1 * cm),
Paragraph('- Chile time is given in number of nights NOT hours, 1 night= 9 hours!!!'),
Paragraph('- Appelaniz: allocated time is based on 15 nights with an average of 9 hours!!!'),
Paragraph('- Once total execution time for Cat. A is at 100%, time will be counted for Cat. B'),
Paragraph('- The 122 h for Suyu are split into 118 h for GROND and 4 h for WFI')    
]
doc.build(flowables)





/home/angela/LaSilla/P115/Statistic/2025-04-09.pdf


### Test for GROND programms, for a specific time period and a specific PID,

In [9]:
########GROND
start = "2025-04-01"
end   = "2025-06-28"
pid_1 = "115.28KF.001"
instr = "GROND"
overhead =  240.0             # new overhead, 3.5 min for fetch... + setup + readout
num= 0
tot_exptime = 0.0
result_1    = eso_query_grond(instr,start,end,pid_1)
num         = num + result_1[0] # number of opt images, optical is not used
print("Number of optical exposures ",result_1[0])
print("Number of NIR exposures",result_1[1])
tot_exptime = tot_exptime + result_1[3]                                  # 41 sec per IR exposure
print(tot_exptime)
tot_time    = (((num/6)*overhead)+ (num*46)+tot_exptime)/3600.0
print("Total execution time [h] =", tot_time)

Number of optical exposures  64
Number of NIR exposures 415
31015.639000000003
Total execution time [h] = 10.144344166666668


### Test for WFI programms, for a specific time period and a specific PID,

In [12]:
########WFI
start = "2025-04-01"
end   = "2025-06-28"
pid_1 = "115.28JZ.002"
instr = "WFI"
overhead =  240.0             # new overhead, 3.5 min for fetch... + setup + readout
num= 0
tot_exptime = 0.0
result_1    = eso_query(instr,start,end,pid_1)
num         = num+result_1[0]
overhead    =   (num*150.0)                           # 19 min overheads per OB (1 f
tot_exptime = (tot_exptime+result_1[1])/3600.0
tot_time    = tot_exptime+(overheads)/3600.0

In [13]:
print(num,tot_exptime)

8 0.7998169444444445
